In [1]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
pip install fast_ml --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv(r"C:\Users\Sahil Patel\Desktop\AV\AV Assignments\dataset\Rain_in_aus_data_scaled_with_feature_extraction.csv")
df.head(3)

Location_Penrith  WindSpeedDiff  Location_Bendigo  Location_Watsonia  \
0             False           37.5             False              False   
1             False           11.0             False              False   
2             False           14.0             False              False   

   Location_Uluru  Location_Perth  Location_Nuriootpa  WindDir3pm_NNE  \
0           False           False               False           False   
1           False           False               False           False   
2           False           False               False           False   

   Location_Albany  WindDir3pm_NNW  ...  WindSpeed3pm.1  Humidity9am  \
0            False           False  ...            22.0     1.510122   
1            False           False  ...             7.0     0.426211   
2            False           False  ...            11.0     1.076558   

   Humidity9am.1   MaxTemp  MaxTemp.1  WindDir3pm_NW  MaxTemp.2  MaxTemp.3  \
0          100.0 -1.648527       10.8          False  -1.648527       10.8   
1           80.0 -0.999320       15.4          False  -0.999320       15.4   
2           92.0 -0.124303       21.6          False  -0.124303       21.6   

   Evaporation  Evaporation.1  
0    -0.591534            3.4  
1    -1.343942            2.0  
2    -0.376560            3.8  

[3 rows x 25 columns]

In [4]:
from fast_ml.model_development import train_valid_test_split
X_train, y_train, X_validation, y_validation, X_test, y_test = train_valid_test_split(df, train_size=0.6, valid_size=0.2, test_size=0.2, target='RainTomorrow')

In [5]:
y_train

34552    0
2406     1
32289    0
31549    0
9608     1
        ..
20158    0
27949    0
34024    0
23936    0
7471     1
Name: RainTomorrow, Length: 24000, dtype: int64

#### Run all the cells above. 

In [6]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Define base estimators
estimators = [
    ('rf', RandomForestClassifier(max_depth=20, min_samples_leaf=10, n_estimators=61, random_state=42, n_jobs=-1)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))),
    ('gb', GradientBoostingClassifier(n_estimators=400, learning_rate=0.3, max_depth=10, random_state=42))
]

In [7]:
from sklearn.model_selection import cross_val_predict
import numpy as np

# Generate out-of-fold predictions for each base model using cross-validation on training data
X_train_meta = []
for name, model in estimators:
    predictions = cross_val_predict(model, X_train, y_train, cv=5, method="predict_proba")
    X_train_meta.append(predictions[:, 1])  # Assuming binary classification     

# Convert to array
X_train_meta = np.column_stack(X_train_meta)

In [8]:
from sklearn.linear_model import LogisticRegression

# Train meta-model (Logistic Regression) on the out-of-fold predictions
meta_model = LogisticRegression()
meta_model.fit(X_train_meta, y_train)

LogisticRegression()

In [9]:
# Function to generate meta features
def generate_meta_features(estimators, X):
    meta_features = []
    for name, model in estimators:
        model.fit(X_train, y_train)  # Train each model on the full training set
        predictions = model.predict_proba(X)
        meta_features.append(predictions[:, 1])  # Assuming binary classification
    return np.column_stack(meta_features)

# Generate meta features for validation and test sets
X_validation_meta = generate_meta_features(estimators, X_validation)
X_test_meta = generate_meta_features(estimators, X_test)

In [10]:
from sklearn.metrics import f1_score

# Predictions on validation and test data using the meta-model
y_pred_validation = meta_model.predict(X_validation_meta)
y_pred_test = meta_model.predict(X_test_meta)

# Calculate and print F1 score for the validation data
f1_score_validation = f1_score(y_validation, y_pred_validation)
print(f"F1 Score for Validation Data: {f1_score_validation}")

# Calculate and print F1 score for the test data
f1_score_test = f1_score(y_test, y_pred_test)
print(f"F1 Score for Test Data: {f1_score_test}")

F1 Score for Validation Data: 0.701595677848976
F1 Score for Test Data: 0.6971857410881801
